In [149]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import datetime
import folium
import os
import seaborn as sns

In [51]:
filename_list = os.listdir("../../data/dacon/kkc1/public_data/train_AT_TSALET_ALL/")
filename_list_2019 = []    
for filename in filename_list :
    if '2019' in filename :
        filename_list_2019.append(filename)
        
path = '../../data/dacon/kkc1/public_data/train_AT_TSALET_ALL/'
df_list = []
for filename in tqdm(filename_list_2019) :
    df = pd.read_csv(path + filename)
    df_list.append(df)
data = pd.concat(df_list, sort=False).reset_index(drop=True)

  0%|          | 0/12 [00:00<?, ?it/s]

# 품목 품종별 주산지 mapping

In [52]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

In [53]:
joosan_dict = dict()

# 품종별 주산지 mapping
for pum in tqdm(unique_pum) :
    pum_df = data[data['PUM_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break
       
# 품종별 주산지 mapping
for pum in tqdm(unique_kind) :
    pum_df = data[data['KIND_NM']==pum]
    joosan_list = pum_df.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False).index
    for i in range(10) :
        if joosan_list[i][-1] in ['군','구','도','시'] : # 국산만 골라내기
            joosan_dict[pum] = joosan_list[i]
            break

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [54]:
joosan_dict

{'배추': '전라남도 해남군',
 '무': '제주도 제주시',
 '양파': '전라남도 무안군',
 '건고추': '전남 해남군',
 '마늘': '경상남도 창녕군',
 '대파': '전라남도 진도군',
 '얼갈이배추': '경기도 포천시',
 '양배추': '제주도 제주시',
 '깻잎': '경상남도 밀양시',
 '시금치': '경기도 포천시',
 '미나리': '경기도 시흥시',
 '당근': '제주도 제주시',
 '파프리카': '강원도 철원군',
 '새송이': '충청남도 천안시',
 '팽이버섯': '경상북도 청도군',
 '토마토': '부산 강서구',
 '청상추': '전북 남원시',
 '백다다기': '충청남도 천안시',
 '애호박': '경상남도 진주시',
 '캠벨얼리': '경상북도 상주시',
 '샤인마스캇': '경상북도 김천시'}

# 주산지 - 관측지점 mapping
# 주산지의 위도 경도 구하기

In [55]:
y = [] #위도
x = [] #경도

Kakao_ServiceKey = open('Kakao_ServiceKey.txt', 'r').read()
headers = {"Authorization": Kakao_ServiceKey}
for san in tqdm(joosan_dict.values()) :
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + san
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    y.append(float(match_first['y'])) #위도
    x.append(float(match_first['x'])) #경도
joosan_xy = pd.DataFrame({'SAN_NM' : joosan_dict.values(), 'y' : y, 'x' : x}).reset_index()

  0%|          | 0/21 [00:00<?, ?it/s]

In [56]:
joosan_xy

,index,SAN_NM,y,x
0,0,전라남도 해남군,34.573517,126.599270
1,1,제주도 제주시,33.499534,126.531171
2,2,전라남도 무안군,34.990341,126.481705
3,3,전남 해남군,34.573517,126.599270
4,4,경상남도 창녕군,35.544615,128.492168
5,5,전라남도 진도군,34.486829,126.263446
6,6,경기도 포천시,37.894993,127.200333
7,7,제주도 제주시,33.499534,126.531171
8,8,경상남도 밀양시,35.503817,128.746608
9,9,경기도 포천시,37.894993,127.200333


# 농업기상관측지점정보

In [57]:
stn_info = pd.read_csv('./RDA_SPOT_INFO.csv')
stn_info['관측시작일'] = pd.to_datetime(stn_info['관측시작일'])
stn_info = stn_info[stn_info['관측시작일'].dt.year<=2015]
#34.990341	126.481705stn_info[stn_info['지점코드'] == 534833E001] ## df[조건식]

In [58]:
stn_info

,도명,지점명,지점코드,위도,경도,고도,설치주소,관측시작일
0,경기도,가평군 가평읍,477802A001,37.84621,127.50063,80.0,경기도 가평군 가평읍 아랫마장길 59,2011-05-22
1,경기도,고양시 구산동,411801A001,37.67453,126.70070,24.0,경기도 고양시 일산서구 구산동 1942,2010-12-06
2,경기도,고양시 덕양구,412040A002,37.64918,126.87036,39.0,경기도 고양시 덕양구 고양대로 1695 (원흥동),2013-08-24
3,경기도,광주시 목현동,464030A001,37.43231,127.23394,91.0,경기도 광주시 이배재로 209-5,2010-11-01
4,경기도,김포시 월곶면,415743A001,37.69489,126.55614,43.0,경기도 김포시 월곶면 오리정로 13,2010-12-14
...,...,...,...,...,...,...,...,...
205,부산,부산시 강서구,618803A001,35.20937,128.96128,3.0,경남 부산시 강서구 대저1동 2038,2010-12-20
206,인천,옹진군 백령면,409911A001,37.95742,124.71063,11.0,인천광역시 옹진군 백령면 진촌리 2359-4,2014-12-18
207,인천,옹진군 영흥면,409871A001,37.25288,126.46034,11.0,인천광역시 옹진군 영흥면 영흥남로435번길 29,2012-05-26
208,울산,울산시 청량면,689861A001,35.52411,129.26704,51.0,울산광역시 울주군 청량면 청량천변로 211,2009-03-02


# 주산지별로 농업기상관측지점과 거리 기준으로 mapping


In [59]:
y_san = joosan_xy['y']
x_san = joosan_xy['x']
y_stn = stn_info['위도']
x_stn = stn_info['경도']
code_dict = dict()

for y_san, x_san, san_name in zip(joosan_xy['y'], joosan_xy['x'], joosan_xy['SAN_NM']) :
    min_distance = 1000 #임의로 초기값 설정
    for y_stn, x_stn, stn_code in zip(stn_info['위도'], stn_info['경도'], stn_info['지점코드']) :
        distance = ((y_san-y_stn)**2 + (x_san-x_stn)**2)**0.5 #거리        
        if distance < min_distance :
            min_distance = distance 
            stn_nearby = str(stn_code)
    code_dict[san_name] = stn_nearby

In [60]:
code_dict

{'전라남도 해남군': '536824B002',
 '제주도 제주시': '063057B009',
 '전라남도 무안군': '534833E001',
 '전남 해남군': '536824B002',
 '경상남도 창녕군': '635821A001',
 '전라남도 진도군': '539823A001',
 '경기도 포천시': '487915A001',
 '경상남도 밀양시': '627911A001',
 '경기도 시흥시': '429843A001',
 '강원도 철원군': '269811A001',
 '충청남도 천안시': '330846A001',
 '경상북도 청도군': '714902A001',
 '부산 강서구': '618803A001',
 '전북 남원시': '590823A001',
 '경상남도 진주시': '660985B001',
 '경상북도 상주시': '742290A001',
 '경상북도 김천시': '037268B004'}

# 농업기상관측지점 및 산지 분포 시각화

In [61]:
stn_location = stn_info[['지점명','위도', '경도']]
san_location = joosan_xy

map = folium.Map(location = [36, 128], zoom_start =7)

# 농업기상관측지점 분포 (파랑)
for index in stn_location.index:
    stn_latitude = stn_location.loc[index,"위도"]
    stn_longtitude = stn_location.loc[index,"경도"]
    stn_tooltip = stn_location.loc[index,'지점명']
    folium.Marker([stn_latitude, stn_longtitude], popup = '('+str(stn_latitude)+', '+str(stn_longtitude)+')', tooltip = stn_tooltip).add_to(map) 

# 산지 분포 (빨강)    
for index in san_location.index:    
    san_latitude = san_location.loc[index,"y"]
    san_longtitude = san_location.loc[index,"x"]
    san_tooltip = san_location.loc[index,'SAN_NM']    
    folium.Marker([san_latitude, san_longtitude], popup = '('+str(san_latitude)+', '+str(san_longtitude)+')', tooltip = san_tooltip, icon = folium.Icon(color = 'red')).add_to(map)
    
map

# 농업기상데이터 API - 월별 일 기본 관측데이터 조회
관측년도, 관측월, 관측지점명, 관측지점코드 값으로 조회
본 대회의 내용으로 운영계정 신청 가능 (신청 후 바로 승인되며, 일일 트래픽 10000 이용가능)
https://www.data.go.kr/iim/api/selectAPIAcountView.do

In [62]:
CropWeather_ServiceKey = open('CropWeather_ServiceKey.txt', 'r').read()
year_list = ['2015','2016', '2017', '2018', '2019', '2020']
month_list = ['01','02','03','04','05','06','07','08','09','10','11','12']
weather = pd.DataFrame()
first_run = 0
code_list = code_dict.values() # 주산지에 mapping 된 지점에 대해서만 조회
year_error, month_error, stn_code_error, url_error, f_obs_date_error = [],[],[],[],[]
colname_dict = dict()

for stn_code in tqdm(code_list) :
    for year in year_list :
        for month in month_list :
            url = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/GnrlWeather/getWeatherMonDayList?'
            params = {
                'serviceKey' : CropWeather_ServiceKey, #인증키
                'Page_No' : '1', # 페이지 번호
                'Page_Size' : '31', # 한 페이지 결과 수(1~100) (31일 이내 전체 표기)
                'search_Year' : year, # 관측년도
                'search_Month' : month, #관측월
                'obsr_Spot_Code' : stn_code # 관측지점코드
            }

            # url에 params 적용하기(붙이기)
            for key, value in zip(params.keys(), params.values()):
                if key == 'serviceKey' :
                    url = url + key +'=' + value
                else :
                    url = url + '&' + key + '=' + value
            
            try :
                response = urllib.request.urlopen(url).read()
                response_string = ET.fromstring(response)

                # response - header(0) / body(1) - ...items(3) - item(0) 
                items = response_string[1][3]

                if first_run == 0 :
                    for i in items[0] :
                        colname_dict[i.tag] = [] # {'no' : [], 'stn_Code' : [], ...} 
                        first_run += 1

                # 일자별로 반복 실행
                num_days = len(items) #28 or 30 or 31
                for index in range(num_days) : 
                    # 해당하는 리스트에 원소 넣기
                    for i in items[index] :
                        colname_dict[i.tag].append(i.text)
            except :
                year_error.append(year)
                month_error.append(month)
                stn_code_error.append(stn_code)
                url_error.append(url)
                f_obs_date_error.append(stn_info[stn_info['지점코드']==stn_code].reset_index()['관측시작일'][0])
               
            
# DataFrame에 값 채워넣기             
for col in colname_dict.keys() :
    weather[col] = colname_dict[col]

# 에러 발생한 요청 모음    
error = pd.DataFrame({'year': year_error, 
                      'month': month_error,
                      'stn_code': stn_code_error,
                      'url': url_error,  
                      '관측시작일': f_obs_date_error})

  0%|          | 0/17 [00:00<?, ?it/s]

# 농업기상관측지점 및 산지 분포 시각화

In [63]:
weather

,no,stn_Code,stn_Name,date,temp,max_Temp,min_Temp,hum,widdir,wind,rain,sun_Time,sun_Qy,condens_Time,gr_Temp,soil_Temp,soil_Wt
0,1,536824B002,해남군 옥천면,2015-01-01,-1.3,0.6,-2.9,80,295.2,2.3,0.8,None,7.8,None,None,3.36,25.9
1,2,536824B002,해남군 옥천면,2015-01-02,0.8,6,-5.3,72,292,1.7,0.8,None,7.9,None,None,3.49,26.9
2,3,536824B002,해남군 옥천면,2015-01-03,-1,6,-7.8,81.4,311,0.4,0,None,10.4,None,None,2.57,26.9
3,4,536824B002,해남군 옥천면,2015-01-04,3.2,11.7,-2.5,85,303,0.4,0,None,9.5,None,None,3.38,26.8
4,5,536824B002,해남군 옥천면,2015-01-05,5.3,12.6,-3.1,87.6,119.1,0.5,6.2,None,7.4,None,None,3.87,26.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36448,27,037268B004,상주시 공성면,2020-12-27,-1.5,4.3,-5.9,86.3,93.6,0.1,0,0,4.2,0,None,3.55,12
36449,28,037268B004,상주시 공성면,2020-12-28,0.6,13,-6.4,81.6,93.5,0.2,0,392,9.5,0,None,3.73,12
36450,29,037268B004,상주시 공성면,2020-12-29,0.1,9.3,-5.6,80.8,278.8,1.3,0,162,6.7,101,None,3.97,12
36451,30,037268B004,상주시 공성면,2020-12-30,-7.6,-1.5,-10.1,64.5,277.4,3.5,0,327,8.7,0,None,4.04,11.9


In [64]:
error

,year,month,stn_code,url,관측시작일
0,2015,01,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
1,2015,02,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
2,2015,03,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
3,2015,04,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
4,2015,05,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
5,2015,06,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
6,2015,07,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
7,2015,08,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
8,2015,09,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01
9,2015,10,063057B009,http://apis.data.go.kr/1390802/AgriWeather/Wea...,2014-05-01


In [65]:
# 중복 제거
weather = weather.drop_duplicates().reset_index(drop=True)

In [66]:
weather

,no,stn_Code,stn_Name,date,temp,max_Temp,min_Temp,hum,widdir,wind,rain,sun_Time,sun_Qy,condens_Time,gr_Temp,soil_Temp,soil_Wt
0,1,536824B002,해남군 옥천면,2015-01-01,-1.3,0.6,-2.9,80,295.2,2.3,0.8,None,7.8,None,None,3.36,25.9
1,2,536824B002,해남군 옥천면,2015-01-02,0.8,6,-5.3,72,292,1.7,0.8,None,7.9,None,None,3.49,26.9
2,3,536824B002,해남군 옥천면,2015-01-03,-1,6,-7.8,81.4,311,0.4,0,None,10.4,None,None,2.57,26.9
3,4,536824B002,해남군 옥천면,2015-01-04,3.2,11.7,-2.5,85,303,0.4,0,None,9.5,None,None,3.38,26.8
4,5,536824B002,해남군 옥천면,2015-01-05,5.3,12.6,-3.1,87.6,119.1,0.5,6.2,None,7.4,None,None,3.87,26.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34258,27,037268B004,상주시 공성면,2020-12-27,-1.5,4.3,-5.9,86.3,93.6,0.1,0,0,4.2,0,None,3.55,12
34259,28,037268B004,상주시 공성면,2020-12-28,0.6,13,-6.4,81.6,93.5,0.2,0,392,9.5,0,None,3.73,12
34260,29,037268B004,상주시 공성면,2020-12-29,0.1,9.3,-5.6,80.8,278.8,1.3,0,162,6.7,101,None,3.97,12
34261,30,037268B004,상주시 공성면,2020-12-30,-7.6,-1.5,-10.1,64.5,277.4,3.5,0,327,8.7,0,None,4.04,11.9


In [67]:
# dtype변환(object --> float)
for col in weather.columns[4:] :
    weather[col] = weather[col].astype(float)

# 전처리 (기상변수로 추가)
30일씩 12쿼터로 나누어서 평균 기온, 평균 습도, 누적 강수량, 이상 기후 누적 일수 등 추가

In [68]:
first_date = datetime.datetime.strptime('2020-09-28', '%Y-%m-%d') - datetime.timedelta(360)
date_list = [] 
for delta in range(360) :
    date = first_date + datetime.timedelta(days = delta)
    date = datetime.datetime.strftime(date, '%Y-%m-%d')
    date_list.append(date)
date_df = pd.DataFrame({'date' : date_list})
train = pd.read_csv('../../data/dacon/kkc1/public_data/train.csv')
print(date_df.shape)
print(train.shape)
train2 = pd.concat([date_df, train], sort = False).reset_index(drop=True) #2015~2020-09-28
# train2 = pd.concat([train], sort = False).reset_index(drop=True) #2015~2020-09-28

(360, 1)
(1733, 44)


In [94]:
train2[359:365]

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
359,2020-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,...,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
362,2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363,2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,...,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
364,2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,...,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0


In [208]:
def weather_feature(temp_df, train2, date_df, allweather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12) :
    # 2015년도 일자 추가
    temp_df = train2[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    
    # 품종과 주산지 날씨 mapping
    joosanji = joosan_dict[pum]
    joosan_code = code_dict[joosanji]
    joosan_weather = allweather[allweather['stn_Code']==joosan_code].reset_index(drop=True)
    end_index = np.where(joosan_weather['date']=='2020-09-28')[0][0]
    joosan_weather = joosan_weather.iloc[:end_index+1] #2020-09-28 까지만 자르기
    temp_df = temp_df.merge(joosan_weather, on='date', how='left')

    # weather feature 추가
#     col_list = temp_df.columns[6:]
    col_list = ['soil_Temp', 'soil_Wt']
    
    for num in range(1,num_quaters+1) :
#     for num in tqdm(range(1,num_quaters+1)) :
        for index in range(360, len(temp_df)) :
            temp_quater_df = temp_df.iloc[index-quater_days*num : index-quater_days*(num-1)] #  
#             quater_temp = temp_quater_df['temp']
#             temp_df.loc[index, f'rain_sum_{num}q'] = temp_quater_df['rain'].sum() # 누적 강수량
#             temp_df.loc[index, f'heavy_rain_count_{num}q'] = np.where(temp_quater_df['rain']>90, 1, 0).sum() # 평균 강수량 90mm 이상 누적 일수
#             temp_df.loc[index, f'low_temp_count_{num}q'] = np.where(quater_temp<5, 1, 0).sum() # 일평균 기온 5도 이하 누적 일수
#             temp_df.loc[index, f'middle_temp_count_{num}q'] = np.where(((quater_temp>15)&(quater_temp<22)), 1, 0).sum() # 일평균 기온 15~22도 누적 일수
#             temp_df.loc[index, f'high_temp_count_{num}q'] = np.where(quater_temp>32, 1, 0).sum() # 일평균 기온 32도 이상 누적 일수
            for col in col_list :
                temp_df.loc[index, f'avg_{col}_{num}q'] = temp_quater_df[col].mean() # 각 기상 요소의 평균값
                
    drop_col_list = temp_df.columns[3:19]
    temp_df = temp_df.drop(drop_col_list, 1).reset_index(drop=True)
    temp_df = temp_df.iloc[360:].reset_index(drop=True)
    
    # p_lag, q_lag 추가
    for lag in range(1,29) :
#     for lag in tqdm(range(1,29)) :
      temp_df[f'p_lag_{lag}'] = -1
      temp_df[f'q_lag_{lag}'] = -1
      for index in range(lag, len(temp_df)) :
        temp_df.loc[index, f'p_lag_{lag}'] = temp_df[f'{pum}_가격(원/kg)'][index-lag] #1일전, 2일전, ... 가격을 feature로 추가
        temp_df.loc[index, f'q_lag_{lag}'] = temp_df[f'{pum}_거래량(kg)'][index-lag] #1일전, 2일전, ... 거래량을 feature로 추가

    # month 추가
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['month'] = temp_df['date'].dt.month   
    
    for week in ['1_week', '2_week', '4_week']:
        temp_df[week] = 0
        n_week = int(week[0])
        for index in range(len(temp_df)):
            try: temp_df[week][index] = temp_df[f'{pum}_가격(원/kg)'][index+7*n_week]
            except: continue   
        
    temp_df = temp_df.iloc[:,3:]
    return temp_df

In [209]:
# weather_feature 함수 예시
pum = '배추'
temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
kkc = weather_feature(temp_df, train2, date_df, weather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12)

In [225]:
kkc.describe()

,avg_soil_Temp_1q,avg_soil_Wt_1q,avg_soil_Temp_2q,avg_soil_Wt_2q,avg_soil_Temp_3q,avg_soil_Wt_3q,avg_soil_Temp_4q,avg_soil_Wt_4q,avg_soil_Temp_5q,avg_soil_Wt_5q,...,p_lag_26,q_lag_26,p_lag_27,q_lag_27,p_lag_28,q_lag_28,month,1_week,2_week,4_week
count,1733.000000,1196.000000,1733.000000,1196.000000,1733.000000,1196.000000,1733.000000,1196.000000,1733.000000,1196.000000,...,1733.000000,1.733000e+03,1733.000000,1.733000e+03,1733.000000,1.733000e+03,1733.000000,1733.000000,1733.000000,1733.000000
mean,15.718702,18.673205,15.720489,18.673205,15.718166,18.673205,15.717625,18.673205,15.714716,18.673205,...,571.412002,1.095140e+06,570.747836,1.094479e+06,570.116561,1.093741e+06,6.280439,595.886324,594.439123,590.532602
std,8.117439,5.785071,8.119640,5.785071,8.116912,5.785071,8.116442,5.785071,8.114986,5.785071,...,385.257883,8.445860e+05,385.251983,8.449948e+05,385.292039,8.453919e+05,3.373900,411.292671,412.648549,415.530962
min,2.518333,3.916667,2.518333,3.916667,2.518333,3.916667,2.518333,3.916667,2.518333,3.916667,...,-1.000000,-1.000000e+00,-1.000000,-1.000000e+00,-1.000000,-1.000000e+00,1.000000,0.000000,0.000000,0.000000
25%,7.900667,14.562500,7.900667,14.562500,7.900667,14.562500,7.900667,14.562500,7.900667,14.562500,...,325.000000,6.679840e+05,324.000000,6.672914e+05,323.000000,6.664825e+05,3.000000,333.000000,331.000000,326.000000
50%,15.730667,19.458333,15.730667,19.458333,15.730667,19.458333,15.730667,19.458333,15.730667,19.458333,...,558.000000,1.050392e+06,557.000000,1.049711e+06,556.000000,1.049286e+06,6.000000,577.000000,577.000000,570.000000
75%,23.157333,23.300000,23.157333,23.300000,23.157333,23.300000,23.152000,23.300000,23.152000,23.300000,...,794.000000,1.378308e+06,794.000000,1.378308e+06,793.000000,1.378308e+06,9.000000,805.000000,805.000000,805.000000
max,29.409667,29.225000,29.409667,29.225000,29.409667,29.225000,29.409667,29.225000,29.409667,29.225000,...,5000.000000,5.210215e+06,5000.000000,5.210215e+06,5000.000000,5.210215e+06,12.000000,5000.000000,5000.000000,5000.000000


In [211]:
import lightgbm
import warnings

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

In [212]:
# preprocessing 함수 예시
# weather_feature 함수 예시
pum = '배추'
temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
# kkc = weather_feature(temp_df, train2, date_df, weather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12)

In [213]:
def nmae(week_answer, week_submission):
#     answer = week_answer.to_numpy()
    answer = week_answer
    target_idx = np.where(answer!=0)
    true = answer[target_idx]
    pred = week_submission[target_idx]
    score = np.mean(np.abs(true-pred)/true)
    
    return score


def at_nmae(pred, dataset):
    y_true = dataset.get_label()
    week_1_answer = y_true[0::3]
    week_2_answer = y_true[1::3]
    week_4_answer = y_true[2::3]
    
    week_1_submission = pred[0::3]
    week_2_submission = pred[1::3]
    week_4_submission = pred[2::3]
    
    score1 = nmae(week_1_answer, week_1_submission)
    score2 = nmae(week_2_answer, week_2_submission)
    score4 = nmae(week_4_answer, week_4_submission)
    
    score = (score1+score2+score4)/3
    
    return 'score', score, False

In [220]:
def model_train(x_train, y_train, x_valid, y_valid) :
    params = {'learning_rate': 0.01, 
              'max_depth': 15, 
              'boosting': 'gbdt', 
              'objective': 'regression',  
              'is_training_metric': True, 
              'num_leaves': 100, 
              'feature_fraction': 0.8, 
              'bagging_fraction': 0.8, 
              'bagging_freq': 5, 
              'seed':42,
              'num_threads':8,
              'verbose':-1
             }

    model = lightgbm.train(params, 
                   train_set = lightgbm.Dataset(data = x_train, label = y_train),
                   num_boost_round = 10000, 
                   valid_sets = lightgbm.Dataset(data = x_valid, label = y_valid), 
                   init_model = None, 
                   early_stopping_rounds = 100,
                   feval = at_nmae,
                   verbose_eval = False
                    )
    
    return model

In [221]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

In [222]:
model_dict = {}
split = 28 #validation

for pum in tqdm(unique_pum + unique_kind):
    # 품목 품종별 전처리
    temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    temp_df = weather_feature(temp_df, train2, date_df, weather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12)
    
    # 주차별(1,2,4w) 학습
    for week_num in [1,2,4] :
        x = temp_df[temp_df[f'{week_num}_week']>0].iloc[:,:-3]
        y = temp_df[temp_df[f'{week_num}_week']>0][f'{week_num}_week']
        
        #train, test split
        x_train = x[:-split]
        y_train = y[:-split]
        x_valid = x[-split:]
        y_valid = y[-split:]
        
        model_dict[f'{pum}_model_{week_num}'] = model_train(x_train, y_train, x_valid, y_valid)

  0%|          | 0/21 [00:00<?, ?it/s]

In [218]:
# def model_train(x_train, y_train, x_valid, y_valid) :
#     params = {'learning_rate': 0.01, 
#               'max_depth': 15, 
#               'boosting': 'gbdt', 
#               'objective': 'regression',  
#               'is_training_metric': True, 
#               'num_leaves': 100, 
#               'feature_fraction': 0.8, 
#               'bagging_fraction': 0.8, 
#               'bagging_freq': 5, 
#               'seed':42,
#               'num_threads':8,
#               'verbose':-1
#              }

#     model = lightgbm.train(params, 
#                    train_set = lightgbm.Dataset(data = x_train, label = y_train),
#                    num_boost_round = 10000, 
#                    valid_sets = lightgbm.Dataset(data = x_valid, label = y_valid), 
#                    init_model = None, 
#                    early_stopping_rounds = 100,
#                    feval = at_nmae,
#                    verbose_eval = False
#                     )
    
#     return model

# split = 28


# # 품목 품종별 전처리
# temp_df = train[['date',f'배추_거래량(kg)', f'배추_가격(원/kg)']]
# temp_df = weather_feature(temp_df, train2, date_df, weather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12)
    
# # 주차별(1,2,4w) 학습

# x = temp_df[temp_df[f'1_week']>0].iloc[:,:-3]
# y = temp_df[temp_df[f'1_week']>0][f'1_week']
       
#         #train, test split
# x_train = x[:-split]
# y_train = y[:-split]
# x_valid = x[-split:]
# y_valid = y[-split:]
        
# model = model_train(x_train, y_train, x_valid, y_valid)

In [219]:
# model.best_score

In [223]:
submission = pd.read_csv('../../data/dacon/kkc1/sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2020')]['예측대상일자'].str.split('+').str[0].unique()
# ['2020-09-29', ...]

for date in tqdm(public_date_list) :
    test = pd.read_csv(f'../../data/dacon/kkc1/public_data/test_files/test_{date}.csv')
    for pum in unique_pum + unique_kind:
        # 예측기준일에 대해 전처리
        temp_test = pd.DataFrame([{'date' : date}]) #예측기준일
        alldata = pd.concat([train, test, temp_test], sort=False).reset_index(drop=True)
        alldata = alldata[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']].fillna(0)
        alldata = alldata.iloc[-28:].reset_index(drop=True)
        
        train3 = pd.concat([date_df, alldata], sort = False).reset_index(drop=True) #2015~2020-09-28
        
        alldata = weather_feature(alldata, train3, date_df, weather, pum, joosan_dict, code_dict, quater_days=30, num_quaters=12)
#         temp_test = alldata.iloc[-1].astype(float)
        temp_test = alldata.iloc[-1].astype(float)[:-3]
        
        # 개별 모델을 활용하여 1,2,4주 후 가격 예측
        for week_num in [1,2,4] :
            temp_model = model_dict[f'{pum}_model_{week_num}']
            result = temp_model.predict(temp_test)
            condition = (submission['예측대상일자']==f'{date}+{week_num}week')
            idx = submission[condition].index
            submission.loc[idx, f'{pum}_가격(원/kg)'] = result[0]

  0%|          | 0/38 [00:00<?, ?it/s]

In [224]:
submission.to_csv('kkc_weather_soil_pq_lag_maxdepth15_numleaves100.csv',index=False)